In [ ]:
import pandas as pd
import os
import json
from googleapiclient.discovery import build
from pymongo import MongoClient

#### Importing videos csv 

In [ ]:
videos_df = pd.read_csv('./csvs/modified_videos_xx.csv')   # last updated videos dataset

In [ ]:
videos_df.info()

#### Getting list of youtube_id for videos missing title in live dataset 


In [ ]:
videos_YTids_noTitle_list = list(videos_df.loc[videos_df['title'].isna(),'youtube_id'])
videos_YTids_noTitle_list[:5]

### YouTube API key

In [ ]:
developer_key = os.environ.get('DEVELOPER_KEY')

youtube = build('youtube', 'v3', developerKey=developer_key) # project-0127 Data Cleaning

#### Fetching title of videos

In [ ]:
part_string = 'snippet'
videos_youtube_id = videos_YTids_noTitle_list

In [ ]:
videos_title_dict = {}
for id in range(0,len(videos_youtube_id)):

    try:
        request = youtube.videos().list(
            part=part_string,
            id=videos_youtube_id[id],
        )
        response = request.execute()

        if response['items']!=[] and response['items'][0]['snippet']['title'] != '':

            videos_title = response['items'][0]['snippet']['title']
        
        else:
            videos_title = None

    except Exception as e:
        print("An exception occured")
        print(e)
        videos_title = None

    videos_title_dict[videos_youtube_id[id]] = videos_title

In [ ]:
title = open('./youtube_api/videos_title.txt',"a")
title.write(json.dumps(videos_title_dict))
title.close()

### Connection to MongoDB

In [ ]:
mongodb_uri = os.environ.get('MONGODB_URI')
database_name = os.environ.get('DATABASE_NAME')

client = MongoClient(mongodb_uri)
database = client[database_name]

#### Connecting to videos collection

In [ ]:
videos = database['modified_videos_xx']   # last updated videos collection

#### Updating video title

In [ ]:
try:
    for (key,value) in videos_title_dict.items():
        videos.update_many(
        {"youtube_id" : key},
        {"$set": { "title" : value}}
        );
except Exception as e:
    print(e)